In [2]:
#@title Set up the notebook
!pip install Biopython
from Bio import SeqIO
import numpy as np
import pandas as pd
import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from collections import Counter
from sklearn import model_selection, linear_model

# import gdown
# data_path = 'https://drive.google.com/uc?id=1f1CtRwSohB7uaAypn8iA4oqdXlD_xXL1'
# cov2_sequences = 'SARS_CoV_2_sequences_global.fasta'
# gdown.download(data_path, cov2_sequences, True)

!wget 'https://storage.googleapis.com/inspirit-ai-data-bucket-1/Data/AI%20Scholars/Sessions%206%20-%2010%20(Projects)/Project%20-%20DNA%20Detectives/SARS_CoV_2_sequences_global.fasta'
cov2_sequences = 'SARS_CoV_2_sequences_global.fasta'

  Using cached https://files.pythonhosted.org/packages/ad/a4/237edd5f5e5b68d9543c79bcd695ef881e6317fbd0eae1b1e53e694f9d54/biopython-1.81.tar.gz
  Running setup.py bdist_wheel for Biopython ... error
  Complete output from command /usr/local/anaconda3/bin/python -u -c "import setuptools, tokenize;__file__='/tmp/pip-install-bbc6h_ty/Biopython/setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" bdist_wheel -d /tmp/pip-wheel-kqw0yu5z --python-tag cp37:
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build/lib.linux-x86_64-3.7
  creating build/lib.linux-x86_64-3.7/Bio
  copying Bio/File.py -> build/lib.linux-x86_64-3.7/Bio
  copying Bio/LogisticRegression.py -> build/lib.linux-x86_64-3.7/Bio
  copying Bio/MarkovModel.py -> build/lib.linux-x86_64-3.7/Bio
  copying Bio/MaxEntropy.py -> build/lib.linux-x86_64-3.7/Bio
  copying Bio/NaiveBayes.py -> build/lib.linux-x86_64-3

Failed to build Biopython
  Running setup.py install for Biopython ... error
    Complete output from command /usr/local/anaconda3/bin/python -u -c "import setuptools, tokenize;__file__='/tmp/pip-install-bbc6h_ty/Biopython/setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" install --record /tmp/pip-record-2bwhclnk/install-record.txt --single-version-externally-managed --compile:
    running install
    running build
    running build_py
    creating build
    creating build/lib.linux-x86_64-3.7
    creating build/lib.linux-x86_64-3.7/Bio
    copying Bio/File.py -> build/lib.linux-x86_64-3.7/Bio
    copying Bio/LogisticRegression.py -> build/lib.linux-x86_64-3.7/Bio
    copying Bio/MarkovModel.py -> build/lib.linux-x86_64-3.7/Bio
    copying Bio/MaxEntropy.py -> build/lib.linux-x86_64-3.7/Bio
    copying Bio/NaiveBayes.py -> build/lib.linux-x86_64-3.7/Bio
    copying Bio/Seq.py -> build/lib.linux-x86

    copying Bio/NMR/NOEtools.py -> build/lib.linux-x86_64-3.7/Bio/NMR
    copying Bio/NMR/__init__.py -> build/lib.linux-x86_64-3.7/Bio/NMR
    copying Bio/NMR/xpktools.py -> build/lib.linux-x86_64-3.7/Bio/NMR
    creating build/lib.linux-x86_64-3.7/Bio/Pathway
    copying Bio/Pathway/__init__.py -> build/lib.linux-x86_64-3.7/Bio/Pathway
    creating build/lib.linux-x86_64-3.7/Bio/Pathway/Rep
    copying Bio/Pathway/Rep/Graph.py -> build/lib.linux-x86_64-3.7/Bio/Pathway/Rep
    copying Bio/Pathway/Rep/MultiGraph.py -> build/lib.linux-x86_64-3.7/Bio/Pathway/Rep
    copying Bio/Pathway/Rep/__init__.py -> build/lib.linux-x86_64-3.7/Bio/Pathway/Rep
    copying Bio/PDB/AbstractPropertyMap.py -> build/lib.linux-x86_64-3.7/Bio/PDB
    copying Bio/PDB/Atom.py -> build/lib.linux-x86_64-3.7/Bio/PDB
    copying Bio/PDB/Chain.py -> build/lib.linux-x86_64-3.7/Bio/PDB
    copying Bio/PDB/DSSP.py -> build/lib.linux-x86_64-3.7/Bio/PDB
    copying Bio/PDB/Dice.py -> build/lib.linux-x86_64-3.7/Bio/PDB
 

Command "/usr/local/anaconda3/bin/python -u -c "import setuptools, tokenize;__file__='/tmp/pip-install-bbc6h_ty/Biopython/setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" install --record /tmp/pip-record-2bwhclnk/install-record.txt --single-version-externally-managed --compile" failed with error code 1 in /tmp/pip-install-bbc6h_ty/Biopython/
You are using pip version 18.1, however version 24.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


ModuleNotFoundError: No module named 'Bio'

In [3]:
sequences = [r for r in SeqIO.parse(cov2_sequences, 'fasta')]
sequence_num =  0#@param {type:"integer"}
print(sequences[sequence_num])

#how many sequences?
n_sequences = len(sequences)
print("There are %f sequences" % n_sequences)

#differences?
sequence_1 = np.array(sequences[1])
sequence_10 = np.array(sequences[10])
similar = np.mean(sequence_1 == sequence_10)
percent_similarity =  100*similar
print("Sequence 1 and 10 similarity: %", percent_similarity)

#visualize mutations w histogram
reference = np.array(sequences[0])
mutations_per_seq = []

for i in range (1,len(sequences)):
  sim = np.sum(reference != sequences[i])
  mutations_per_seq.append(sim)
mutations_per_seq = np.array(mutations_per_seq)

plt.hist(mutations_per_seq)
plt.xlabel('# mutations')
plt.ylabel('# sequences')
plt.show()

min_number_of_mutations  =  300#@param {type:"integer"}
idx = np.random.choice(np.where(mutations_per_seq>min_number_of_mutations)[0])
print("Sequence %i has > %.0f mutations \n" % (idx, min_number_of_mutations))
print(sequences[idx], '\n')
print("The sequence is composed of: ")
Counter(np.array(sequences[idx]))

NameError: name 'SeqIO' is not defined

In [ ]:
# Extract Features
# Note: This can take a couple minutes to run! 
# but we can monitor our progress using the tqdm library
mutation_df = pd.DataFrame()
n_bases_in_seq = len(sequences[0])

# Iterate though all positions in this sequence.
for location in tqdm.tqdm(range(n_bases_in_seq)): # tqdm is a nice library that prints our progress.
  bases_at_location = np.array([s[location] for s in sequences])
  # If there are no mutations at this position, move on.
  if len(set(bases_at_location))==1: continue # If
  for base in ['A', 'T', 'G', 'C', '-']:
    feature_values = (bases_at_location==base)
    
    # Set the values of any base that equals 'N' to np.nan.
    feature_values[bases_at_location==['N']] = np.nan
    
    # Convert from T/F to 0/1.
    feature_values  = feature_values*1
    
    # Make the column name look like <location>_<base> (1_A, 2_G, 3_A, etc.)
    column_name = str(location) + '_' + base
    mutation_df[column_name] = feature_values

# Print the size of the feature matrix/table.
n_rows = np.shape(mutation_df)[0]
n_columns = np.shape(mutation_df)[1]
print("Size of matrix: %i rows x %i columns" %(n_rows, n_columns))

# Check what the matrix looks like:
mutation_df.tail()

In [ ]:
countries_to_regions_dict = {
         'Australia': 'Oceania',
         'China': 'Asia',
         'Hong Kong': 'Asia' ,
         'India': 'Asia' ,
         'Nepal': 'Asia' ,
         'South Korea': 'Asia' ,
         'Sri Lanka': 'Asia' ,
         'Taiwan': 'Asia' ,
         'Thailand': 'Asia' ,
         'USA': 'North America' ,
         'Viet Nam': 'Asia' 
}

regions = [countries_to_regions_dict[c] if c in 
           countries_to_regions_dict else 'NA' for c in countries]
mutation_df['label'] = regions

In [ ]:
#Balacing data between samples from each region
#countries to regions
countries_to_regions_dict = {
         'Australia': 'Oceania',
         'China': 'Asia',
         'Hong Kong': 'Asia' ,
         'India': 'Asia' ,
         'Nepal': 'Asia' ,
         'South Korea': 'Asia' ,
         'Sri Lanka': 'Asia' ,
         'Taiwan': 'Asia' ,
         'Thailand': 'Asia' ,
         'USA': 'North America' ,
         'Viet Nam': 'Asia' 
}

regions = [countries_to_regions_dict[c] if c in 
           countries_to_regions_dict else 'NA' for c in countries]
mutation_df['label'] = regions

#balancing 
balanced_df = mutation_df.copy()
balanced_df['label'] = regions
balanced_df = balanced_df[balanced_df.label!='NA']
balanced_df = balanced_df.drop_duplicates()
samples_north_america = balanced_df[balanced_df.label== 'North America' ]
samples_oceania = balanced_df[balanced_df.label== 'Oceania' ]
samples_asia = balanced_df[balanced_df.label== 'Asia' ]

# Number of samples we will use from each region.
n = min(len(samples_north_america),
        len(samples_oceania),
        len(samples_asia))

balanced_df = pd.concat([samples_north_america[:n],
                    samples_asia[:n],
                    samples_oceania[:n]])
print("Number of samples in each region: ", Counter(balanced_df['label']))